## Dataset
AI-Hub: https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=598

power shell을 이용해 src -r [로컬 파일 경로] [유저 이름]@[ip주소]:[서버 경로] 로 파일 업로드

### 어노테이션 포멧 분석

#### class 개수 파악

In [ ]:
import os
import json

# JSON 파일들이 있는 폴더 경로
json_folder_path = '/home/elicer/jyj/ms_parking'

# 모든 JSON 파일에서 label과 class 정보를 추출하는 함수
def count_labels_and_classes(json_folder_path):
    bbox2d_class_count = {}
    segmentation_class_count = {}

    for root, dirs, files in os.walk(json_folder_path):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as f:
                    data = json.load(f)

                    # bbox2d 내의 "name"을 추출하여 bbox2d_class_count에 저장
                    for bbox in data.get("bbox2d", []):
                        class_name = bbox.get("name")
                        if class_name:
                            bbox2d_class_count[class_name] = bbox2d_class_count.get(class_name, 0) + 1

                    # segmentation 내의 "name"을 추출하여 segmentation_class_count에 저장
                    for seg in data.get("segmentation", []):
                        class_name = seg.get("name")
                        if class_name:
                            segmentation_class_count[class_name] = segmentation_class_count.get(class_name, 0) + 1

    return bbox2d_class_count, segmentation_class_count

# 함수 호출 및 결과 출력
bbox2d_class_count, segmentation_class_count = count_labels_and_classes(json_folder_path)

# 각 class별로 빈도 출력
print("bbox2d Classes and Counts:")
for class_name, count in bbox2d_class_count.items():
    print(f'Class: {class_name}, Count: {count}')

print("\nsegmentation Classes and Counts:")
for class_name, count in segmentation_class_count.items():
    print(f'Class: {class_name}, Count: {count}')

# 총 클래스 수 출력
print(f"\nTotal bbox2d Classes: {len(bbox2d_class_count)}")
print(f"Total segmentation Classes: {len(segmentation_class_count)}")


### Segmentation 데이터 파악

ex) TL2실내중소형주차장 파일

In [ ]:
base_path = u'/home/elicer/jyj/ms_parking'
folders = [f'ms_parking_{str(i).zfill(3)}' for i in range(1, 57)]
label_folders = [os.path.join(base_path, folder, 'label') for folder in folders]

# 모든 JSON 데이터를 저장할 리스트
all_data = []

# 각 폴더의 label 폴더 안에 있는 모든 JSON 파일을 읽어서 합치기
for label_folder in label_folders:
    for json_file in os.listdir(label_folder):
        if json_file.endswith('.json'):
            json_path = os.path.join(label_folder, json_file)
            with open(json_path, 'r') as f:
                data = json.load(f)
                all_data.append(data)

In [ ]:
# 합쳐진 데이터를 데이터프레임으로 변환
df = pd.DataFrame(all_data)
df

In [ ]:
empty_segmentation = df[df['segmentation'].apply(lambda x: len(x) != 0)]
print(empty_segmentation)

#### 바운딩 박수 2D, 3D, segmentation 데이터 개수 추출

In [ ]:
# 모든 bbox2d, bbox3d, segmentation 데이터를 저장할 리스트
bbox2d_data = []
bbox3d_data = []
segmentation_data = []

# 각 폴더의 label 폴더 안에 있는 모든 JSON 파일을 읽어서 합치기
for label_folder in label_folders:
    for json_file in os.listdir(label_folder):
        if json_file.endswith('.json'):
            json_path = os.path.join(label_folder, json_file)
            with open(json_path, 'r') as f:
                data = json.load(f)

                # bbox2d 데이터 추출
                for bbox2d in data.get('bbox2d', []):
                    bbox2d_entry = {
                        'image': data['image'],
                        'name': bbox2d.get('name'),
                        'occluded': bbox2d.get('occluded'),
                        'bbox': bbox2d.get('bbox'),
                        'status': bbox2d.get('status', None)  # 'status'가 있을 경우에만 가져옴
                    }
                    bbox2d_data.append(bbox2d_entry)

                # segmentation 데이터 추출
                for seg in data.get('segmentation', []):
                    seg_entry = {
                        'image': data['image'],
                        'name': seg.get('name'),
                        'size' : seg.get('size'),
                        'pattern' : seg.get('pattern'),
                        'polygon': seg.get('polygon'),
                    }
                    segmentation_data.append(seg_entry)

In [ ]:
df_bbox2d = pd.DataFrame(bbox2d_data)
df_bbox3d = pd.DataFrame(bbox3d_data)
df_segmentation = pd.DataFrame(segmentation_data)

#### 바운딩 박스 클래스 개수 파악

In [ ]:
print(len(df_bbox2d['name'].unique()))

In [ ]:
df_bbox2d['name'].value_counts()

#### 바운딩 박스 개수 시각화

In [ ]:
name_counts = df_bbox2d['name'].value_counts()

plt.figure(figsize=(8, 5))
name_counts.plot(kind='bar')
plt.title('BBox2d Object Counts by Name')
plt.xlabel('Object Name')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

#### Bounding Box와 Segmentation 시각화

In [ ]:
name_counts_bbox2d = df_bbox2d['name'].value_counts()
name_counts_segmentation = df_segmentation['name'].value_counts()

# 두 개의 시리즈를 하나의 데이터프레임으로 결합
name_counts_combined = pd.DataFrame({
    'bbox2d': name_counts_bbox2d,
    'segmentation': name_counts_segmentation
}).fillna(0)  # NaN 값을 0으로 채우기

# 합계로 정렬하기 위해 합계를 새로운 열로 추가
name_counts_combined['total'] = name_counts_combined.sum(axis=1)

# 총합을 기준으로 내림차순 정렬
name_counts_combined = name_counts_combined.sort_values(by='total', ascending=False)

# 총합 열 제거 (그래프에 필요 없으므로)
name_counts_combined = name_counts_combined.drop(columns=['total'])

# 그래프 그리기
plt.figure(figsize=(12, 8))
name_counts_combined.plot(kind='bar')

# 그래프 제목 및 축 레이블 설정
plt.title('Counts by Name (bbox2d and segmentation)')
plt.xlabel('Object/Segmentation Name')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')

# 레이아웃 조정 및 그래프 출력
plt.tight_layout()
plt.show()


#### 이미지별 Segmentation 개수 파악

In [ ]:
import os
import pandas as pd
import json

# 기본 경로 설정
base_path = "/home/elicer/jyj/중소형주차장_segmentation_only"
directories = os.listdir(base_path)

# DataFrame 초기화
df = pd.DataFrame(columns=["Parking Space Count", "Driveable Space Count"])

# 모든 노상 폴더 순회
for dir_name in directories:
    dir_path = os.path.join(base_path, dir_name, "label")
    
    if os.path.isdir(dir_path):
        files = os.listdir(dir_path)
        
        # 모든 JSON 파일 순회
        for file_name in files:
            if file_name.endswith(".json"):
                file_path = os.path.join(dir_path, file_name)
                
                # JSON 파일 읽기
                with open(file_path, 'r', encoding='utf-8') as file:
                    data = json.load(file)
                
                # Parking Space와 Driveable Space의 개수를 세는 변수
                parking_space_count = 0
                driveable_space_count = 0
                
                # Segmentation 리스트를 순회하며 각 공간의 이름을 확인
                for segment in data.get('segmentation', []):
                    if segment['name'] == 'Parking Space':
                        parking_space_count += 1
                    elif segment['name'] == 'Driveable Space':
                        driveable_space_count += 1
                
                # 행 이름 설정 (노상 번호와 파일 이름)
                row_name = f"{dir_name}_{file_name}"
                
                # DataFrame에 추가
                df.loc[row_name] = [parking_space_count, driveable_space_count]

In [ ]:
pd.DataFrame(parking_zero_list)

In [ ]:
pd.DataFrame(driveable_zero_list)

### 데이터셋 전처리

#### Segmentation 정보가 있는 json 파일만 추출

In [ ]:
import os
import json

# JSON 파일들이 있는 폴더 경로
json_folder_path = '/home/elicer/jyj/중소형주차장'
output_folder_path = '/home/elicer/jyj/중소형주차장_segmentation_only'

# 새로운 JSON 파일로 저장하는 함수 및 크기 비교
def save_segmentation_only_json(json_folder_path, output_folder_path):
    # 출력 폴더가 없으면 생성
    os.makedirs(output_folder_path, exist_ok=True)

    original_total_size = 0
    reduced_total_size = 0

    for root, dirs, files in os.walk(json_folder_path):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                original_total_size += os.path.getsize(file_path)  # 원본 파일 크기 합산

                with open(file_path, 'r') as f:
                    data = json.load(f)

                    # segmentation이 있는 경우에만 파일 생성
                    if "segmentation" in data and data["segmentation"]:
                        # 불필요한 항목들 제거
                        data.pop("bbox2d", None)
                        data.pop("calib", None)
                        data.pop("bbox3d", None)

                        # 새로운 JSON 파일로 저장
                        relative_path = os.path.relpath(root, json_folder_path)
                        output_dir = os.path.join(output_folder_path, relative_path)
                        os.makedirs(output_dir, exist_ok=True)
                        output_file_path = os.path.join(output_dir, file)
                        with open(output_file_path, 'w') as output_file:
                            json.dump(data, output_file, indent=4)
                        
                        print(f'파일 생성됨: {output_file_path}')
                        reduced_total_size += os.path.getsize(output_file_path)  # 변환된 파일 크기 합산

    return original_total_size, reduced_total_size

# 함수 호출
original_size, reduced_size = save_segmentation_only_json(json_folder_path, output_folder_path)

# 크기 비교 출력
reduction_percentage = 100 * (original_size - reduced_size) / original_size if original_size > 0 else 0
print(f"원본 데이터셋 크기: {original_size / (1024 * 1024):.2f} MB")
print(f"축소된 데이터셋 크기: {reduced_size / (1024 * 1024):.2f} MB")
print(f"데이터셋 축소 비율: {reduction_percentage:.2f}%")


#### 라벨데이터로 이미지데이터 축소

ex) TL2실내중소형주차장 파일

In [ ]:
import os
import shutil
import json

# JSON 파일들이 있는 폴더 경로
json_folder_path = r"C:\elicetrain\088.주차공간탐색을위한차량관점복합데이터\01.데이터\1.Training\라벨링데이터\실내\중소형주차장_segmentation_only"
image_source_folder = r"C:\elicetrain\088.주차공간탐색을위한차량관점복합데이터\01.데이터\1.Training\원천데이터\실내\중소형주차장"
image_destination_folder = r"C:\elicetrain\088.주차공간탐색을위한차량관점복합데이터\01.데이터\1.Training\원천데이터\실내\중소형주차장_segmentation"

# JSON 파일과 매칭되는 이미지 파일을 로컬에 복사하는 함수
def copy_matching_images(json_folder_path, image_source_folder, image_destination_folder):
    # 출력 폴더가 없으면 생성
    os.makedirs(image_destination_folder, exist_ok=True)

    for root, dirs, files in os.walk(json_folder_path):
        for file in files:
            if file.endswith('.json'):
                json_file_path = os.path.join(root, file)
                print(f'Processing JSON file: {json_file_path}')  # 디버깅 출력을 추가
                try:
                    with open(json_file_path, 'r') as f:
                        data = json.load(f)

                        # 이미지 파일명을 JSON에서 추출하면서 모든 공백 제거
                        image_file_name = data.get("image", "").strip().replace(" ", "")
                        print(f'Extracted image file name: {image_file_name}')  # 디버깅 출력을 추가

                        # 경로에서 추가적으로 필요한 정보를 붙임
                        if image_file_name:
                            # JSON 파일의 부모 디렉토리 이름 가져오기
                            json_parent_dir = os.path.basename(os.path.dirname(root))

                            # 이미지의 실제 경로 구성
                            image_source_path = os.path.join(image_source_folder, json_parent_dir, image_file_name)
                            image_destination_path = os.path.join(image_destination_folder, json_parent_dir, image_file_name)

                            print(f'Source image path: {image_source_path}')  # 디버깅 출력을 추가
                            print(f'Destination image path: {image_destination_path}')  # 디버깅 출력을 추가

                            # 이미지 파일이 존재하면 복사
                            if os.path.exists(image_source_path):
                                os.makedirs(os.path.dirname(image_destination_path), exist_ok=True)
                                shutil.copy2(image_source_path, image_destination_path)
                                print(f'이미지 파일 복사됨: {image_source_path} -> {image_destination_path}')
                            else:
                                print(f'이미지 파일이 존재하지 않음: {image_source_path}')
                        else:
                            print('Image file name is empty or not found in JSON')

                except json.JSONDecodeError as e:
                    print(f"JSONDecodeError in file {json_file_path}: {e}")
                except Exception as e:
                    print(f"An error occurred with file {json_file_path}: {e}")

# 함수 호출
copy_matching_images(json_folder_path, image_source_folder, image_destination_folder)

print("모든 이미지 파일이 성공적으로 복사되었습니다.")

#### 데이터셋 이름 정렬 및 병합

In [ ]:
import os
import shutil

# 원본 폴더 경로 설정
#image_source_root = '/home/elicer/jyj/원천데이터'
#json_source_root = '/home/elicer/jyj/라벨데이터'

# 대상 폴더 경로 설정
image_destination_folder = '/home/elicer/dataset/mergerd_images' # 이미지 파일 복사 대상 경로
json_destination_folder = '/home/elicer/dataset/mergerd_labels'   # JSON 파일 복사 대상 경로

# 대상 폴더가 존재하지 않으면 생성
if not os.path.exists(image_destination_folder):
    os.makedirs(image_destination_folder)

if not os.path.exists(json_destination_folder):
    os.makedirs(json_destination_folder)

# 인덱스 초기화
index = 1

# 이미지 파일 복사 및 이름 변경
for root, dirs, files in os.walk(image_source_root):
    for filename in files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
            source_file = os.path.join(root, filename)
            new_filename = f"{index:08}.jpg"
            destination_file = os.path.join(image_destination_folder, new_filename)
            shutil.copy2(source_file, destination_file)
            index += 1

# JSON 파일 복사 및 이름 변경
index = 1  # 인덱스 초기화 (이미지와 동일한 인덱스를 사용하려면 이 부분을 제거)
for root, dirs, files in os.walk(json_source_root):
    for filename in files:
        if filename.lower().endswith('.json'):
            source_file = os.path.join(root, filename)
            new_filename = f"{index:08}.json"
            destination_file = os.path.join(json_destination_folder, new_filename)
            shutil.copy2(source_file, destination_file)
            index += 1

print("이미지 및 JSON 파일 복사가 완료되었습니다.")



#### Json 파일 내 image 파일 이름을 변경된 이름으로 변경

In [ ]:
import os
import json

# 경로 설정
#label_dir = "/home/elicer/dataset/mergerd_labels"

# 라벨 디렉토리의 모든 파일을 순회
for filename in os.listdir(label_dir):
    # 파일 경로
    filepath = os.path.join(label_dir, filename)
    
    # 파일이 json 파일인 경우에만 처리
    if filename.endswith(".json"):
        # json 파일 열기
        with open(filepath, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # 라벨 파일 이름에서 확장자를 제거한 후 .jpg를 추가
        image_name = os.path.splitext(filename)[0] + ".jpg"
        
        # 기존의 "image": "Camera/ 00000000 .jpg"를 새로운 이미지 이름으로 수정
        data["image"] = image_name
        
        # 수정된 내용을 파일에 다시 쓰기
        with open(filepath, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)

print("모든 파일이 성공적으로 수정되었습니다.")

#### json 파일을 coco 데이터셋 구조로 변경

In [ ]:
import os
import json

# 기본 경로 (상위 폴더)
#base_path = '/home/elicer/dataset/mergerd_labels'  # 실제 경로로 변경
#output_base_path = '/home/elicer/dataset/coco_mergerd_labels'  # 변환된 파일을 저장할 경로로 변경

# 변환 작업을 위한 함수
def convert_to_coco_format(input_json_path, output_json_path, image_id_start, annotation_id_start):
    with open(input_json_path, 'r') as f:
        data = json.load(f)

    image_id = image_id_start
    annotation_id = annotation_id_start

    coco_format = {
        "images": [],
        "annotations": [],
        "categories": []
    }

    category_id_map = {
        # "Car": 1,
        # "Other Vehicle": 2,
        "Parking Space": 1,
        "Driveable Space": 2,
        # "Traffic Cone": 5,
        # "Traffic Pole": 6  # 새로운 범주 추가
    }

    coco_format["categories"] = [
        # {"id": 1, "name": "Car"},
        # {"id": 2, "name": "Other Vehicle"},
        {"id": 1, "name": "Parking Space"},
        {"id": 2, "name": "Driveable Space"},
        # {"id": 5, "name": "Traffic Cone"},
        # {"id": 6, "name": "Traffic Pole"}
    ]

    image_info = {
        "file_name": f"{str(image_id_start).zfill(8)}.png",  # 이미지 파일명
        "height": 1080,  # 실제 이미지의 높이로 변경
        "width": 1920,  # 실제 이미지의 너비로 변경
        "id": image_id
    }
    coco_format["images"].append(image_info)

    # for bbox in data["bbox2d"]:
    #     category_id = category_id_map.get(bbox["name"])
    #     if category_id is None:
    #         category_id = max(category_id_map.values()) + 1
    #         category_id_map[bbox["name"]] = category_id
    #         coco_format["categories"].append({"id": category_id, "name": bbox["name"]})

    #     annotation = {
    #         "id": annotation_id,
    #         "image_id": image_id,
    #         "category_id": category_id,
    #         "bbox": bbox["bbox"],
    #         "area": (bbox["bbox"][2] - bbox["bbox"][0]) * (bbox["bbox"][3] - bbox["bbox"][1]),  # bbox의 면적
    #         "iscrowd": 0
    #     }
    #     coco_format["annotations"].append(annotation)
    #     annotation_id += 1

    for seg in data["segmentation"]:
        category_id = category_id_map.get(seg["name"])
        if category_id is None:
            category_id = max(category_id_map.values()) + 1
            category_id_map[seg["name"]] = category_id
            coco_format["categories"].append({"id": category_id, "name": seg["name"]})

        annotation = {
            "id": annotation_id,
            "image_id": image_id,
            "category_id": category_id,
            "segmentation": [sum(seg["polygon"], [])],  # 평탄화된 polygon list
            "area": 0,  # segmentation area 필요 시 계산
            "iscrowd": 0
        }
        coco_format["annotations"].append(annotation)
        annotation_id += 1

    with open(output_json_path, 'w') as f:
        json.dump(coco_format, f, indent=4)  # indent=4로 지정하여 들여쓰기 적용

    return annotation_id

# 모든 하위 폴더를 순회하면서 변환 수행
image_id_counter = 1
annotation_id_counter = 1

for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith('.json'):
            input_json_path = os.path.join(root, file)
            relative_path = os.path.relpath(root, base_path)
            output_dir = os.path.join(output_base_path, relative_path)
            os.makedirs(output_dir, exist_ok=True)
            output_json_path = os.path.join(output_dir, file)

            annotation_id_counter = convert_to_coco_format(
                input_json_path, output_json_path, image_id_counter, annotation_id_counter
            )

            image_id_counter += 1

print("모든 파일이 변환되었습니다.")

#### 이미지 데이터 train, validation 분할

In [ ]:
import os
import shutil
import random

# 설정
#image_dir = '/home/elicer/dataset/merged_images'  # 이미지 파일이 있는 폴더
#coco_dir = '/home/elicer/dataset/merged_coco'   # 라벨 파일이 있는 폴더
#output_dir = '/home/elicer/dataset'  # 결과물 저장 폴더
train_ratio = 0.8  # Train/Val 비율 설정 (예: 0.8 = 80% train, 20% val)

# 결과물 저장 폴더 생성
train_image_dir = os.path.join(output_dir, 'images/train')
train_label_dir = os.path.join(output_dir, 'labels/train')
val_image_dir = os.path.join(output_dir, 'images/val')
val_label_dir = os.path.join(output_dir, 'labels/val')

os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

# 이미지 파일 목록 가져오기
image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
random.shuffle(image_files)  # 파일 목록을 랜덤으로 섞기

# Train/Val 분할
train_count = int(len(image_files) * train_ratio)
train_files = image_files[:train_count]
val_files = image_files[train_count:]

# 파일 복사 함수
def move_files(files, image_output_dir, label_output_dir):
    for image_file in files:
        # 이미지 파일 복사
        shutil.copy2(os.path.join(image_dir, image_file), image_output_dir)

        # 해당 이미지의 라벨 파일 복사
        label_file = os.path.splitext(image_file)[0] + '.json'
        label_path = os.path.join(coco_dir, label_file)
        
        if os.path.exists(label_path):
            shutil.copy2(label_path, label_output_dir)

# Train 세트 처리
move_files(train_files, train_image_dir, train_label_dir)

# Val 세트 처리
move_files(val_files, val_image_dir, val_label_dir)

print("Train/Val 세트로 성공적으로 분할되었습니다.")

#### 분할된 image 데이터에 맞춰 json 파일도 분할

In [ ]:
import os
import shutil

# 경로 설정
#train_image_folder = '/home/elicer/project_data/train'
#val_image_folder = '/home/elicer/project_data/val'
#label_folder = '/home/elicer/jm/data/label'

# 라벨 파일 확장자
label_extension = '.json'

# 파일 이동 함수 정의
def move_labels(image_folder, dest_folder):
    os.makedirs(dest_folder, exist_ok=True)
    
    for filename in os.listdir(image_folder):
        file_base, _ = os.path.splitext(filename)
        label_filename = file_base + label_extension
        label_path = os.path.join(label_folder, label_filename)
        
        if os.path.exists(label_path):
            shutil.move(label_path, os.path.join(dest_folder, label_filename))

# 라벨을 train 폴더로 이동
move_labels(train_image_folder, train_image_folder)

# 라벨을 val 폴더로 이동
move_labels(val_image_folder, val_image_folder)

print("라벨 파일이 이미지 경로에 맞게 분리되었습니다.")

### 전처리 과정 문제 해결

#### 라벨과 이미지 매칭 테스트

In [ ]:
# label test 코드
import os
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json
from PIL import Image
import numpy as np

# 이미지 및 라벨 파일 경로 지정
image_dir = "/home/elicer/jyj/원천데이터/mergerd_2실외중소형주차장_300--"
label_dir = "/home/elicer/jyj/라벨데이터_축소/mergerd_2실외중소형주차장_300--"

# 이미지 파일 리스트 생성
image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg") or f.endswith(".png")]

# 랜덤으로 세 개의 파일 선택
random_files = random.sample(image_files, 3)

def visualize_segmentation(image_path, json_path):
    """
    주어진 이미지와 JSON 파일을 시각화하는 함수.
    """
    with open(json_path) as f:
        labels = json.load(f)

    # 이미지 로드
    image = Image.open(image_path)
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)

    # 라벨 시각화
    for label in labels['segmentation']:
        color = 'cyan'  # 라벨에 대한 색상을 설정 (임의로 선택)
        polygon = label['polygon']
        polygon = np.array(polygon)
        p = patches.Polygon(polygon, closed=True, fill=False, edgecolor=color, linewidth=2)
        ax.add_patch(p)

    plt.title(f"Segmentation for {os.path.basename(image_path)}")
    plt.show()

# 선택된 파일들에 대한 작업 수행
for filename in random_files:
    image_path = os.path.join(image_dir, filename)
    json_path = os.path.join(label_dir, os.path.splitext(filename)[0] + ".json")
    
    print(f"Processing image: {filename}")  # 이미지 파일 이름 출력
    
    if os.path.exists(json_path):
        visualize_segmentation(image_path, json_path)
    else:
        print(f"Label file not found for {filename}")

#### 폴더 내 파일 개수 파악

In [ ]:
# 파일 개수 파악 코드
import os
import math

def count_files_and_size_in_folder(folder_path):
    file_count = 0
    total_size = 0  # 총 파일 크기 (바이트 단위)
    
    for root, dirs, files in os.walk(folder_path):
        file_count += len(files)
        for file in files:
            file_path = os.path.join(root, file)
            total_size += os.path.getsize(file_path)
    
    return file_count, total_size

def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return f"{s} {size_name[i]}"

# 예시 폴더 경로
folder_path = "/home/elicer/merged_labels"  # 확인하려는 폴더 경로로 바꿔주세요

# 파일 수 및 총 용량 계산
total_files, total_size = count_files_and_size_in_folder(folder_path)
formatted_size = convert_size(total_size)

print(f"폴더 내의 파일 수: {total_files}개")
print(f"폴더 내의 총 파일 용량: {formatted_size}")

#### 파일 디렉토리 구조 비교 후 다른 부분을 출력

In [ ]:
# 파일 디렉토리 구조 비교 코드
import os

def get_files_and_directories(root_dir):
    file_structure = {}
    for dirpath, dirnames, filenames in os.walk(root_dir):
        relative_path = os.path.relpath(dirpath, root_dir)
        file_structure[relative_path] = set(filenames)
    return file_structure

def compare_directories(dir1, dir2, output_file):
    files_in_dir1 = get_files_and_directories(dir1)
    files_in_dir2 = get_files_and_directories(dir2)

    all_dirs = set(files_in_dir1.keys()).union(set(files_in_dir2.keys()))

    with open(output_file, 'w', encoding='utf-8') as f:
        for directory in all_dirs:
            files1 = files_in_dir1.get(directory, set())
            files2 = files_in_dir2.get(directory, set())

            if files1 != files2:
                f.write(f"\n차이가 있는 디렉토리: {directory}\n")
                if files1 - files2:
                    f.write(f"{dir1}에만 있는 파일:\n")
                    for file in files1 - files2:
                        f.write(f" - {os.path.join(dir1, directory, file)}\n")
                if files2 - files1:
                    f.write(f"{dir2}에만 있는 파일:\n")
                    for file in files2 - files1:
                        f.write(f" - {os.path.join(dir2, directory, file)}\n")

if __name__ == "__main__":
    dir1 = "/home/elicer/jyj/라벨데이터_축소/1실내중소형주차장"
    dir2 = "/home/elicer/new_dataset/labels"
    output_file = "directory_comparison_result.txt"

    compare_directories(dir1, dir2, output_file)
    print(f"비교 결과가 {output_file} 파일에 저장되었습니다.")